# Simulation of different circuit perparation of GKP codes

In [1]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt
import optimization
import evaluation
import auxiliary
import pandas as pd
import time
import strawberryfields as sf

/Users/etiennestock/Documents/packages/strawberryfields/strawberryfields/apps/data/sample.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
# gates
def Scgate(phi): return sf.ops.Sgate(1, phi)
def Dcgate(phi): return sf.ops.Dgate(1, phi)
def Vconst(theta): return so.ops.Vgate(np.sign(theta)*1)
def Vtanhgate(s): return sf.ops.Vgate(np.tanh(s))
def V4tanhgate(s): return sf.ops.Vkgate(np.tanh(s), 4)
def V5gate(s): return sf.ops.Vkgate(s, 5)
def V5tanhgate(s): return sf.ops.Vkgate(np.tanh(s), 5)
def V6tanhgate(s): return sf.ops.Vkgate(np.tanh(s), 6)
def V7tanhgate(s): return sf.ops.Vkgate(np.tanh(s), 7)
def V8tanhgate(s): return sf.ops.Vkgate(np.tanh(s), 8)
def V9tanhgate(s): return sf.ops.Vkgate(np.tanh(s), 9)
def V11tanhgate(s): return sf.ops.Vkgate(np.tanh(s), 11)
def Kdl2c2gate(kappa): return sf.ops.Kdlcgate(kappa, 2, 2)
def Kdl6c2gate(kappa): return sf.ops.Kdlcgate(kappa, 6, 2)
def Kdl10c2gate(kappa): return sf.ops.Kdlcgate(kappa, 10, 2)

In [3]:
# hyperparameters of the current run
cutoffs    = [30, 60, 90]
delta      = 0.25 
num_trials = 2 
n_jobs     = 2 
state      = [0, 0] 
num_blocks = [35, 40, 45] 
circuits   = ['XRVconstS', 'XRVconstS', 'XRVconstS']

epsilon = delta**2 
gates   = [auxiliary.string_to_gateset(circuit) for circuit in circuits] 

In [4]:
# arrays to store data for plots and error analysis 
opt = []
fids = []
bestFids = []
bestNorm = []
timeDiff = []

# parallel optimization
for i in range(len(circuits)): 
    opt.append(optimization.Optimization(gates=gates[i], circuit=circuits[i], cutoff=cutoffs[i], delta=delta, epsilon=epsilon, num_blocks=num_blocks[i], num_trials=num_trials, state=state))
    time1 = time.clock_gettime(0)
    fid, _ = opt[i].parallel_optimize_circuit()   
    timeDiff.append(time.clock_gettime(0) - time1)
    
    fids.append(fid)
    bestFids.append(opt[i].get_bestFid())
    bestNorm.append(opt[i].norm_best_state())
    
    print(f'{circuits[i]} optimization done in {int(opt[i].get_time()//60)} minutes and {opt[i].get_time()%60:.0f} seconds.')
    print(f'fidelity: {fids[i]}')
    print(f'norm: {bestNorm[i]}')
    print(f'consistency check: {opt[i].check_fidelity()}')

XRVconstS optimization done in 14 minutes and 57 seconds.
fidelity: [0.97003377 0.98938464]
norm: 0.9902053678855558
consistency check: True
XRVconstS optimization done in 17 minutes and 34 seconds.
fidelity: [0.88005492 0.87915031]
norm: 0.9351400058418357
consistency check: True
XRVconstS optimization done in 39 minutes and 1 seconds.
fidelity: [0.68582179 0.82952005]
norm: 0.9154520893378738
consistency check: True


In [5]:
# save data in folder data
for i in range(len(circuits)): 
    file_name = f"data_{auxiliary.state_dict[tuple(state)]}_{circuits[i]}_{cutoffs[i]}_{str(delta).replace('.', '')}_{num_blocks[i]}"
    print(i)
    opt[i].save(file_name)

0
1
2


In [7]:
# add results
results = pd.DataFrame()
results['circuit'] = circuits
results[r'$n_{\text{cutoff}}$'] = cutoffs
results[r'$n_{params}$'] = np.array([auxiliary.count_gates(circuit) for circuit in circuits])*np.array(num_blocks)
results[r'$|\langle\Psi_g|\Psi_g\rangle|^2$'] = bestNorm
results[r'$|\langle\Psi_g|\Psi_t\rangle|^2$'] = bestFids
results['running time'] = timeDiff
results

,circuit,$n_{\text{cutoff}}$,$n_{params}$,$|\langle\Psi_g|\Psi_g\rangle|^2$,$|\langle\Psi_g|\Psi_t\rangle|^2$,running time
0,XRVconstS,30,140,0.990205,0.989385,897.167207
1,XRVconstS,60,160,0.935140,0.880055,1054.349869
2,XRVconstS,90,180,0.915452,0.829520,2341.249794


In [8]:
# Plot probability distribution of generated and target GKP states
qnum = 8
qlim = qnum*np.sqrt(np.pi)
qvalues = np.linspace(-qlim,qlim, 1000)

ket_GKP = evaluation.prepare_GKP(epsilon=epsilon, cutoff=cutoff, state=state).ket()
wf_GKP = evaluation.wavefunction(q=qvalues, ket=ket_GKP)
gauss = lambda x: np.max(np.abs(wf_GKP*wf_GKP.conj())) * np.exp(-x**2*(delta)**2)

for i in range(len(circuits)):
    ket_rec = opt[i].prepare_best_state().ket()
    wf_rec = evaluation.wavefunction(q=qvalues, ket=ket_rec)
    plt.plot(qvalues, np.abs(wf_rec*wf_rec.conj()), label=f'{circuits[i]}: '+r'$|\langle\Psi_g|\Psi_t\rangle|^2$'+f' = {bestFids[i]:.3f}', linestyle='dotted', linewidth=2)

plt.plot(qvalues, np.abs(wf_GKP*wf_GKP.conj()), label='target', color='purple', linewidth=0.8)
plt.plot(qvalues, gauss(qvalues), label=r'Gaussian envelope with $\Delta =$'+f'{delta}', color='black', linewidth=0.5)

plt.ylabel("$|\Psi(q)|^2$")
plt.xticks(np.sqrt(np.pi)*np.arange(-qnum, qnum), np.arange(-qnum, qnum))
plt.xlabel("$q/\sqrt{\pi}$")
plt.xlim(np.min(qvalues), np.max(qvalues))
plt.ylim(0, np.max(np.abs(wf_GKP*wf_GKP.conj()))+0.1)
plt.title(f'Circuit with {np.max(num_blocks)} preparation rounds and cutoff {cutoff}')
plt.legend()

plt.savefig(f'plots/plot_{cutoff}_{str(delta).replace(".", "")}_{np.max(num_blocks)}_' + '_'.join(str(c) for c in circuits) + '.png')
plt.show()

<>:18: SyntaxWarning: invalid escape sequence '\P'
<>:20: SyntaxWarning: invalid escape sequence '\s'
<>:18: SyntaxWarning: invalid escape sequence '\P'
<>:20: SyntaxWarning: invalid escape sequence '\s'
/var/folders/6q/7zspg0xj68g0zq09v2tt_hyh0000gn/T/ipykernel_1593/2438886067.py:18: SyntaxWarning: invalid escape sequence '\P'
/var/folders/6q/7zspg0xj68g0zq09v2tt_hyh0000gn/T/ipykernel_1593/2438886067.py:20: SyntaxWarning: invalid escape sequence '\s'
/var/folders/6q/7zspg0xj68g0zq09v2tt_hyh0000gn/T/ipykernel_1593/2438886067.py:18: SyntaxWarning: invalid escape sequence '\P'
/var/folders/6q/7zspg0xj68g0zq09v2tt_hyh0000gn/T/ipykernel_1593/2438886067.py:20: SyntaxWarning: invalid escape sequence '\s'


NameError: name 'cutoff' is not defined

In [ ]:
# # calculate the Knill-Glancy error
# n    = 30 
# size = 30 

# results[r'$P_{\text{error}}$'] = np.empty(len(circuits), dtype=np.float32)
# for i in range(len(circuits)): 
#     ket_rec = opt[i].prepare_best_state().ket()
#     wf_rec = evaluation.wavefunction(q=qvalues, ket=ket_rec)
#     results.loc[i, r'$P_{\text{error}}$'] = evaluation.error_gancy_knill(wf_rec, n, size)

# results